# 1. Introduction

## 1.1. Présentation du projet et explication de la démarche


## 1.2. Chargement des packages

In [106]:
import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import Point
from pynsee.geodata import get_geodata_list, get_geodata, GeoFrDataFrame

from outils import *

from matplotlib import pyplot as plt
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from io import BytesIO


## 1.3. Chargement des données

### 1.3.1. Données INSEE
On commence par charger la base de données 'données locales' qui contient entre autre
* Le nom des communes française
* Leur population au dernier recensement
* Leur coordonnées géographiques des communes
On s'aide pour cela de la documentation de pynsee (les commandes ici sont utilisable sans clé API)

In [202]:
# On récupère la liste géographique
geodata_list = get_geodata_list()
# On récupère la liste des limites géographique des départements
com = get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:commune')

mapcom_raw = gpd.GeoDataFrame(com).set_crs("EPSG:3857")

#Le jeu de données contenant les informations que nous cherchons est le suivant
mapcom_raw.sample(5)

100%|██████████| 99/99 [00:31<00:00,  3.11it/s]


,id,nom,nom_m,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,geometry,crsCoord
25992,COMMUNE_0000000009730928,Bertoncourt,BERTONCOURT,08062,Commune simple,126,11,2,08,44,200043156,"MULTIPOLYGON (((487473.588 6366581.491, 487529...",EPSG:3857
7595,COMMUNE_0000000009752646,Vouthon,VOUTHON,16421,Commune simple,410,19,1,16,75,200068914,"MULTIPOLYGON (((50441.948 5722646.46, 50440.16...",EPSG:3857
9937,COMMUNE_0000000009761227,Gourvieille,GOURVIEILLE,11166,Commune simple,73,01,1,11,76,200035855,"MULTIPOLYGON (((200025.209 5363926.922, 200186...",EPSG:3857
23148,COMMUNE_0000002200276536,Entre-Vignes,ENTRE-VIGNES,34246,Commune simple,2160,12,3,34,76,243400520,"MULTIPOLYGON (((454899.252 5425427.886, 454893...",EPSG:3857
29948,COMMUNE_0000000009738252,Crépey,CREPEY,54143,Commune simple,384,10,4,54,44,245400510,"MULTIPOLYGON (((669410.311 6194786.709, 669350...",EPSG:3857


In [199]:
# Surtout dans les DROM-COM certaines communes apparaissent aussi comme sous-préféctures,par exemple:
mapcom_raw[(mapcom_raw['nom_m'] == 'SAINT-CLAUDE')]


,id,nom,nom_m,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,geometry,crsCoord
6,COMMUNE_0000001169858685,Saint-Claude,SAINT-CLAUDE,97124,Commune simple,10700,06,1,971,01,249710070,"MULTIPOLYGON (((-6869802.488 1804329.757, -687...",EPSG:3857
27957,COMMUNE_0000000009749074,Saint-Claude,SAINT-CLAUDE,39478,Sous-préfecture,8727,14,3,39,27,200026573,"MULTIPOLYGON (((648695.631 5837666.044, 648666...",EPSG:3857


In [204]:
# Ainsi, certaines communes apparaisent en double en tant que sous-préféctures, on ne garde ici que les communes

# On identifie les doublons dans la colonne 'nom_m'
duplicates = mapcom_raw[mapcom_raw['nom_m'].duplicated(keep=False)]

# On sélectionne les lignes où 'statut' est égal à 'Commune simple'
simple_communes = duplicates[duplicates['statut'] == 'Commune simple']

# On conserve les lignes avec des 'nom_m' uniques ou celles avec 'Commune simple' lorsque des doublons existent
mapcom = pd.concat([
    mapcom_raw[~mapcom_raw['nom_m'].duplicated(keep=False)],  # Lignes sans doublons
    simple_communes  # Lignes où 'Commune simple' est sélectionné
]).drop_duplicates(subset='nom_m', keep='first')  # S'assurer que 'nom_m' est unique dans le résultat final

# Réinitialiser l'index pour plus de clarté 
mapcom.reset_index(drop=True, inplace=True)

# Afficher le DataFrame final
mapcom.sample(5)


,id,nom,nom_m,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,geometry,crsCoord
29293,COMMUNE_0000000009737298,Foulcrey,FOULCREY,57229,Commune simple,151,21,5,57,44,200068146,"MULTIPOLYGON (((761329.242 6213575.476, 761491...",EPSG:3857
25368,COMMUNE_0000000009746977,Montagny-lès-Seurre,MONTAGNY-LES-SEURRE,21424,Commune simple,113,04,1,21,27,242101509,"MULTIPOLYGON (((582846.804 5947965.234, 583439...",EPSG:3857
24936,COMMUNE_0000000009748086,Ruffey-sur-Seille,RUFFEY-SUR-SEILLE,39471,Commune simple,764,03,2,39,27,200069615,"MULTIPOLYGON (((609453.326 5893212.62, 609406....",EPSG:3857
26171,COMMUNE_0000000009736393,Ménil-aux-Bois,MENIL-AUX-BOIS,55333,Commune simple,45,08,2,55,44,245500327,"MULTIPOLYGON (((602743 6240983.106, 602755.54 ...",EPSG:3857
28092,COMMUNE_0000000009746118,Narbief,NARBIEF,25421,Commune simple,90,14,3,25,27,242504355,"MULTIPOLYGON (((746980.107 5963410.922, 746755...",EPSG:3857


### 1.3.2. Données Géod'air

Les données Géod'air sont un jeu de données issue de Géod'air le pourvoyeur public de données sur la pollution atmosphériques. Elles consistent en des jeux de données .csv tékéchargées préalablements représentant la mesure de 5 polluants différents et des informations conjointes (nom et lieux des sites de meusures, types de mesures, etc.) regroupés par intervalles de 6 mois entre le 1er janvier 2020 et le 31 décembre 2023.
On compte donc au total 40 fichiers .csv.

**Remarque :** Pour obtenir les données une API existe, cependant celle ci ne permet que d'obtenir des données par intervals de 1 jour, afin de gagner du temps dans les requêtes et pour ne pas risquer de surcharger les serveurs de Géod'air nous avons préféré utiliser leur outil de chargement sur-mesure des données

In [225]:
folder_paths = [
   r"CSV_data\GEODAIR\NO2\NO2_moy_hor_per", #Utiliser soit r"..." soit des \\ sinon les "\N" de "\NO2" posent problème
    "CSV_data\GEODAIR\O3\O3_moy_hor_per",
    "CSV_data\GEODAIR\PM2.5\PM2.5_moy_jour_per",
    "CSV_data\GEODAIR\PM10\PM10_moy_jour_per",
    "CSV_data\GEODAIR\SO2\SO2_moy_hor_per"
    ]

#Ici l'instruction "range(8,1,-1)" permet de classer les chemins du plus récent au moins récent.
all_file_paths = [[ folder + str(i) + ".csv" for i in range(8,0,-1)] for folder in folder_paths]

# On combine les 40 (5x8) jeux de données en 5 dataframes allant de janvier 2020 à décembre 2023
# Pour les polluants dans l'ordre suivant: NO2, O3, PM2.5, PM10, SO2
all_combined_df = [pd.concat([pd.read_csv(file, sep = ";") for file in all_file_paths[i]]) for i in range(5)]

#On obtien une liste de 5 dataframes, soit un par polluant

In [8]:
# On souhaite avoir un jeu de données pour lesquels seuls les mesures ayant mesuré tous les polluants sont conservés
# On précise les colonnes sur lesquels on applique la jonction
join_columns = ["Date de début", "Date de fin", "code site"]

# On sélectionne le data frame avec le plus petit nombre de lignes
smallest_df = min(all_combined_df, key=len)

# On initialise avec les plus petit data frame
result_df = smallest_df

# On joint chaque dataframe de notre liste "all_combined_df" avec un suffixe unique
for i, df in enumerate(all_combined_df):
    if df is not smallest_df:  # pour éviter l'auto-jonction
        result_df = pd.merge(
            result_df, 
            df, 
            on=join_columns, 
            how='inner', 
            suffixes=(None, f'_df{i}')  # Un suffixe unique pour chaque data frame
        )

# result_df contient maintenant le inner join of de tous les data frames sur les colonnes spécifées
# On obtient un objet qui contient beaucoup plus de variable que ce qui nous intéresse, on vien donc procéder à un premier tri

# Soit la liste des variables les plus utiles, on peut l'utiliser pour extraire un version synthétique du de result_df
best_variables = [
    'Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Latitude', 'Longitude',
    'Polluant', "type d'influence", 'type de valeur', 'valeur', 'valeur brute', 'unité de mesure','code qualité', 'validité',
    'Polluant_df0', 'type de valeur_df0', 'valeur_df0', 'valeur brute_df0', 'unité de mesure_df0','code qualité_df0', 'validité_df0',
    'Polluant_df1', 'type de valeur_df1', 'valeur_df1', 'valeur brute_df1', 'unité de mesure_df1','code qualité_df1', 'validité_df1',
    'Polluant_df2', 'type de valeur_df2', 'valeur_df2', 'valeur brute_df2', 'unité de mesure_df2','code qualité_df2', 'validité_df2',
    'Polluant_df3', 'type de valeur_df3', 'valeur_df3', 'valeur brute_df3', 'unité de mesure_df3','code qualité_df3', 'validité_df3',
]

best_df = result_df[best_variables]

# Etape cosmétique pour faciliter la lecture des données intermédiaires
# On définit une fonction pour renommer les colonnes
# Définir les colonnes qui doivent garder leur nom tel quel
columns_to_keep = [
    'Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 
    'code site', 'nom site', "type d'implantation", 'Latitude', 'Longitude'
]

# Définir la fonction pour renommer les colonnes
def rename_columns(column_name):
    # Vérifier si la colonne doit conserver son nom
    if column_name in columns_to_keep:
        return column_name
    # Ajouter le suffixe "_SO2" pour les colonnes sans suffixe qui ne sont pas dans columns_to_keep
    elif not any(column_name.endswith(suffixe) for suffixe in ["_df0", "_df1", "_df2", "_df3"]):
        return f"{column_name}_SO2"
    # Remplacer les suffixes spécifiques
    elif column_name.endswith("_df0"):
        return column_name.replace("_df0", "_NO2")
    elif column_name.endswith("_df1"):
        return column_name.replace("_df1", "_O3")
    elif column_name.endswith("_df2"):
        return column_name.replace("_df2", "_PM2.5")
    elif column_name.endswith("_df3"):
        return column_name.replace("_df3", "_PM10")
    else:
        return column_name

best_df = best_df.rename(columns=rename_columns)


# Pour l'instant les données ne sont pas dans un format géographique, on transforme donc le dataframe en GeoDataFrame
best_dfgeo = gpd.GeoDataFrame(
    best_df, geometry=gpd.points_from_xy(best_df.Longitude, best_df.Latitude)
)

# Une visualisation des données donne donc:
best_dfgeo.sample(5)

,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Latitude,Longitude,...,code qualité_PM2.5,validité_PM2.5,Polluant_PM10,type de valeur_PM10,valeur_PM10,valeur brute_PM10,unité de mesure_PM10,code qualité_PM10,validité_PM10,geometry
12870,2021/11/27 00:00:00,2021/11/27 23:59:59,ATMO NORMANDIE,FR28ZAR01,ZAR LE-HAVRE,FR05090,Le Havre ville-haute,Urbaine,49.514686,0.100645,...,A,1,PM10,Moy. journalière,13.0,12.641667,µg-m3,A,1,POINT (0.10064 49.51469)
13307,2021/12/27 00:00:00,2021/12/27 23:59:59,ATMO GRAND EST,FR44ZRE01,ZR GRAND-EST,FR30033,Jonville en Woevre,Rurale régionale,49.065830,5.785556,...,A,1,PM10,Moy. journalière,9.0,9.445833,µg-m3,A,1,POINT (5.78556 49.06583)
4602,2023/05/08 00:00:00,2023/05/08 23:59:59,ATMO HAUTS DE FRANCE,FR32ZRE01,ZR HAUTS-DE-FRANCE,FR18042,P. Roth St Quentin,Urbaine,49.851334,3.284000,...,A,1,PM10,Moy. journalière,9.0,9.308333,µg-m3,A,1,POINT (3.284 49.85133)
5628,2022/07/10 00:00:00,2022/07/10 23:59:59,ATMO REUNION,FR04ZAR01,ZAR SAINT-DENIS,FR38020,Plateau Caillou,Urbaine,-21.022112,55.266773,...,A,1,PM10,Moy. journalière,10.0,10.075000,µg-m3,A,1,POINT (55.26677 -21.02211)
8543,2022/02/01 00:00:00,2022/02/01 23:59:59,ATMO REUNION,FR04ZAR01,ZAR SAINT-DENIS,FR38020,Plateau Caillou,Urbaine,-21.022112,55.266773,...,N,-1,PM10,Moy. journalière,19.0,18.712500,µg-m3,N,-1,POINT (55.26677 -21.02211)


In [205]:
#On revient maintenant à notre liste de dataframe, afin de pouvoir identifier les communes au sein desquelles les mesures ont été faites
#Pour rappel, on a tous les df dans la liste all_combined_df
result = []
result_simple = []
result_simple_agg = []
geometries = []
variable_gdf = []

# On séléctionne les variables les plus intéréssantes pour le projet 
variable_interet = ['Date de fin',
                    "type d'implantation",
                    "type d'influence", 
                    'valeur', 
                    'geometry',
                    'index_right', 
                    'id', 
                    'nom', 
                    'nom_m', 
                    'insee_com', 
                    'statut',
                    'population', 
                    'insee_can', 
                    'insee_arr', 
                    'insee_dep', 
                    'insee_reg',
                    'siren_epci', 
                    'crsCoord'
                    ]

for indice_polluant in range(5):
    gdf = gpd.GeoDataFrame(all_combined_df[indice_polluant], geometry=gpd.points_from_xy(all_combined_df[indice_polluant].Longitude, all_combined_df[indice_polluant].Latitude)).set_crs('EPSG:4326') #EPSG:4326 est le CRS correspondant aux longitudes-latitudes
    #On projette mapcom les données dans le même CRS
    mapcom = mapcom.to_crs(gdf.crs) 
    #Jonction spatiale, l'argument within identifie les coordonnées qui sont au sein des géométries des communes
    result.append(gpd.sjoin(gdf, mapcom, how="left", predicate="within"))
    # Cette étape permet de formater les dates au format datetime
    result[indice_polluant]['Date de fin'] = result[indice_polluant]['Date de fin'].apply(format_date)

    result_simple.append(result[indice_polluant][variable_interet])

    #On va maintenant grouper les mesures par commune et par jour pour alléger le jeu de données
    result_simple_agg.append(result_simple[indice_polluant].groupby(['nom_m', 'Date de fin'], as_index=False).agg({'valeur': 'mean'}))
    
    # On ajoute ici une géométrie en reprenant la première géométrie de chaque groupe
    geometries.append(result_simple[indice_polluant].groupby(['nom_m', 'Date de fin'])['geometry'].first().reset_index())
    
    # On fusionne pour réassocier la géométrie
    variable_gdf.append(gpd.GeoDataFrame(result_simple_agg[indice_polluant].merge(geometries[indice_polluant],on=['nom_m', 'Date de fin']),geometry='geometry', crs=result_simple[indice_polluant].crs))
    
    # On renomme 'Date de fin' en un plus simple 'date'
    variable_gdf[indice_polluant].rename(columns = {'Date de fin' : 'date'}, inplace = True)

# On obtient pour chaque polluant un GéoDataFrame ayant les données suivantes: 
# nom de la commune en majuscule ('nom_m')
# date au format datetime ('date')
# La valeur moyenne des mesures de pulluants ce jour là et à cette date là ('valeur')
# Les coordonnées sous forme d'un objet POINT ('geometry')
variable_gdf[4].sample(5)

#Remarque : cette cellule peut prendre du temps à s'effectuer

,nom_m,date,valeur,geometry
51333,MARTIGUES,2020-12-04,5.725,POINT (5.04273 43.41666)
35634,JONVILLE-EN-WOEVRE,2023-11-06,1.200,POINT (5.78556 49.06583)
20049,FONTAINEBLEAU,2021-01-07,1.400,POINT (2.6453 48.3547)
35302,JONVILLE-EN-WOEVRE,2022-12-08,1.200,POINT (5.78556 49.06583)
77214,SAINT-ETIENNE-DE-MONTLUC,2022-10-06,2.200,POINT (-1.78316 47.28318)


In [219]:
# Enfin, on vient fusionner les 5 dataframe en un (outer join car on veut conserver le plus d'information possible)
all_variable_gdf1 = pd.merge(variable_gdf[0],variable_gdf[1], how='outer', on =['nom_m','date','geometry'], validate='1:1', suffixes = ('_NO2','_O3'))
all_variable_gdf2 = pd.merge(variable_gdf[2],variable_gdf[3], how='outer', on =['nom_m','date','geometry'], validate='1:1', suffixes = ('_PM2.5','_PM10'))
all_variable_gdf2 = pd.merge(all_variable_gdf2,variable_gdf[4], how='outer', on =['nom_m','date','geometry'], validate='1:1', suffixes = (False,'_SO2'))
all_variable_gdf = pd.merge(all_variable_gdf1,all_variable_gdf2, how='outer', on =['nom_m','date','geometry'], validate='1:1', suffixes = (False,False))
mapcom_simple = mapcom[['id', 'nom_m', 'insee_com', 'statut', 'population', 'insee_dep']]
all_variable_gdf = pd.merge(all_variable_gdf, mapcom_simple, how = 'left', on = 'nom_m', validate = 'm:1',suffixes = (False,False))


#Etape cosmétique pour faciliter la lisibilité (on fait apparaître le nom de chaque polluant)
all_variable_gdf.rename(columns = {
    'valeur_NO2': 'NO2',
    'valeur_O3': 'O3',
    'valeur_PM2.5': 'PM2.5',
    'valeur_PM10':'PM10',
    'valeur' : 'SO2'}, inplace = True)


In [242]:
# On vient éliminer certains duplicatats issus d'irrégularités au sein de la variable geometry
grouping_columns = all_variable_gdf.drop(columns=['NO2', 'O3', 'PM2.5', 'PM10', 'SO2','geometry']).columns
#On enlève la colonne geometry car celle ci comporte des doublons à un 100 000ième de coordonnée près
polluants_gdf = all_variable_gdf.groupby(list(grouping_columns), as_index=False).agg({
    'NO2': 'max',
    'O3': 'max',
    'PM2.5': 'max',
    'PM10': 'max',
    'SO2': 'max'
})

#On rajoute la colonne geometry en ne conservant qu'une seulle valeur pour chaque ville
geometries = all_variable_gdf[['nom_m','geometry']].drop_duplicates(subset='nom_m')

polluants_gdf = gpd.GeoDataFrame(pd.merge(polluants_gdf,geometries, how = 'left', on= 'nom_m'),geometry='geometry', crs='EPSG:4326')

In [243]:
polluants_gdf

,nom_m,date,id,insee_com,statut,population,insee_dep,NO2,O3,PM2.5,PM10,SO2,geometry
0,AGDE,2020-01-01,COMMUNE_0000000009761167,34003,Commune simple,29103,34,23.2,53.4,NaN,NaN,NaN,POINT (3.50484 43.2875)
1,AGDE,2020-01-02,COMMUNE_0000000009761167,34003,Commune simple,29103,34,32.8,33.9,NaN,NaN,NaN,POINT (3.50484 43.2875)
2,AGDE,2020-01-03,COMMUNE_0000000009761167,34003,Commune simple,29103,34,24.6,55.1,NaN,NaN,NaN,POINT (3.50484 43.2875)
3,AGDE,2020-01-04,COMMUNE_0000000009761167,34003,Commune simple,29103,34,11.4,67.2,NaN,NaN,NaN,POINT (3.50484 43.2875)
4,AGDE,2020-01-05,COMMUNE_0000000009761167,34003,Commune simple,29103,34,26.7,67.8,NaN,NaN,NaN,POINT (3.50484 43.2875)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560652,XONRUPT-LONGEMER,2023-12-27,COMMUNE_0000000009741120,88531,Commune simple,1488,88,1.7,77.8,NaN,NaN,NaN,POINT (7.01111 48.05112)
560653,XONRUPT-LONGEMER,2023-12-28,COMMUNE_0000000009741120,88531,Commune simple,1488,88,1.7,73.2,NaN,NaN,NaN,POINT (7.01111 48.05112)
560654,XONRUPT-LONGEMER,2023-12-29,COMMUNE_0000000009741120,88531,Commune simple,1488,88,1.4,71.8,NaN,NaN,NaN,POINT (7.01111 48.05112)
560655,XONRUPT-LONGEMER,2023-12-30,COMMUNE_0000000009741120,88531,Commune simple,1488,88,0.8,74.4,NaN,NaN,NaN,POINT (7.01111 48.05112)


### 1.3.3. Données MétéoFrance

In [72]:
# On charge d'abord les données de mesure depuis l'environnement local (comme pour Géod'air)
path = 'CSV_data\METEOFRANCE'
df_meteo_month = []

for year in range(2020, 2024):  
    for month in range(1, 13):  
        date = f"{year}{month:02d}"  # Format "202001", "202002", etc
        file_name = f"synop.{date}.csv.gz"
        file_path = os.path.join(path, file_name)
        try:
            df = pd.read_csv(file_path, compression='gzip', sep=';', encoding='utf-8')
            df_meteo_month.append(df)  # Ajouter chaque DataFrame à la liste
        except FileNotFoundError:
            print(f"Fichier non trouvé : {file_path}")
        except Exception as e:
            print(f"Erreur lors du chargement de {file_path} : {e}")

df_meteo = pd.concat(df_meteo_month, ignore_index=True)

# Etape cosmétique pour ne garder que les variables d'intérêt et simplifier la lecture
colonnes_a_garder = ['numer_sta', 'date', 'pmer', 'tend', 'dd', 'ff', 't', 'u', 'vv', 'rr24']
noms_colonnes = {
    'numer_sta': 'ID',
    'pmer': 'pression_niveau_mer_Pa',
    'tend': 'var_pression_3h_Pa',
    'dd': 'direction_vent_moyen_10mn_deg',
    'ff': 'vitesse_vent_moyen_10mn_m/s',
    't': 'temperature_K',
    'u': 'humidite_%',
    'vv': 'visibilite_horizontale_m',
    'rr24': 'precipitations_24h_mm'
}

def filtre_mesure_minuit(df):
    # Convertir la colonne date au format datetime
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H%M%S', errors='coerce')
    
    # Filtrer les lignes correspondant à minuit (heure, minute et seconde égales à 0)
    df_minuit = df[(df['date'].dt.hour == 0) & 
                   (df['date'].dt.minute == 0) & 
                   (df['date'].dt.second == 0)]
    
    return df_minuit


df_meteo = df_meteo[colonnes_a_garder]
df_meteo = df_meteo.rename(columns=noms_colonnes)
df_meteo = filtre_mesure_minuit(df_meteo)  # Garder seulement les mesures à minuit
df_meteo['date'] = df_meteo['date'].dt.strftime('%Y-%m-%d')

df_meteo.sample(5)

,ID,date,pression_niveau_mer_Pa,var_pression_3h_Pa,direction_vent_moyen_10mn_deg,vitesse_vent_moyen_10mn_m/s,temperature_K,humidite_%,visibilite_horizontale_m,precipitations_24h_mm
644720,81408,2023-09-17,101230,190,110,1.100000,300.450000,79,mq,0.000000
230897,7643,2021-04-30,100990,30,170,1.700000,286.550000,93,16620,14.500000
117104,7558,2020-09-03,102090,60,20,4.000000,286.350000,71,20000,0.000000
287675,7591,2021-08-28,mq,40,30,1.000000,284.250000,83,mq,0.200000
512209,7591,2022-12-16,mq,60,40,3.600000,275.250000,96,mq,27.600000


In [ ]:
# On extrait les données des stations de mesure et on les joint à notre dataframe df_meteo
stations_gdf = gpd.read_file('CSV_data\METEOFRANCE\postesSynop.json')
stations_gdf['Longitude'] = stations_gdf['Longitude'].astype(float)
stations_gdf['Latitude'] = stations_gdf['Latitude'].astype(float)
stations_gdf['ID'] = stations_gdf['ID'].astype(int)

stations_gdf.sample(5)

,ID,Nom,Latitude,Longitude,Altitude,geometry
0,7005,ABBEVILLE,50.136000,1.834000,69,POINT (1.834 50.136)
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.0975 50.57)
2,7020,PTE DE LA HAGUE,49.725167,-1.939833,6,POINT (-1.93983 49.72517)
3,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.45617 49.18)
4,7037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.18167 49.383)
...,...,...,...,...,...,...
57,81401,SAINT LAURENT,5.485500,-54.031667,5,POINT (-54.03167 5.4855)
58,81405,CAYENNE-MATOURY,4.822333,-52.365333,4,POINT (-52.36533 4.82233)
59,81408,SAINT GEORGES,3.890667,-51.804667,6,POINT (-51.80467 3.89067)
60,81415,MARIPASOULA,3.640167,-54.028333,106,POINT (-54.02833 3.64017)


In [73]:
meteo_gdf = gpd.GeoDataFrame(
    pd.merge(df_meteo,stations_gdf, how = 'left', on='ID', validate = 'm:m', suffixes = (False,False)),
    geometry='geometry')

meteo_gdf.sample(5)
#On a vérifié qu'il n'y avait pas d'errerus d'appairement

,ID,date,pression_niveau_mer_Pa,var_pression_3h_Pa,direction_vent_moyen_10mn_deg,vitesse_vent_moyen_10mn_m/s,temperature_K,humidite_%,visibilite_horizontale_m,precipitations_24h_mm,Nom,Latitude,Longitude,Altitude,geometry
640,7690,2020-01-11,102630,50,330,5.100000,281.450000,76,40000,1.000000,NICE,43.648833,7.209000,2,POINT (7.209 43.64883)
55854,78897,2022-08-03,101450,130,90,2.800000,300.450000,79,27640,-0.100000,LE RAIZET AERO,16.264000,-61.516333,11,POINT (-61.51633 16.264)
25225,7790,2021-02-28,102580,50,240,1.800000,278.250000,94,11770,mq,BASTIA,42.540667,9.485167,10,POINT (9.48517 42.54067)
75894,7299,2023-07-04,101630,-10,190,1.200000,291.550000,74,58320,-0.100000,BALE-MULHOUSE,47.614333,7.510000,263,POINT (7.51 47.61433)
48895,7434,2022-04-09,101550,310,280,1.900000,277.450000,84,60000,9.700000,LIMOGES-BELLEGARDE,45.861167,1.175000,402,POINT (1.175 45.86117)


### 1.3.4. Données ADEME

Ce jeu de données est *l'inventaire de gaz à effet de serre territorialisé* de l'ADEME.
Il s'agit du résultat de l'inventaire national de gaz à effet de serre (GES).
D'après l'ADEME : *"La résolution spatiale est communale, structures stables dans le temps et ensuite agrégeables par EPCI. Il est établi à partir à la fois à partir d’une décomposition des émissions nationales de GES au niveau communal et d’informations déjà spatialisées."*

In [68]:
#La première étape est une importation classique avec pandas
inventaire_ges = pd.read_csv("https://data.ademe.fr/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/full")

#On fait apparaître la collonne des totaux et log-totaux (La distribution des totaux suit une loi log-normale)
inventaire_ges['total'] = inventaire_ges.drop(['INSEE commune', 'Commune','lat','lon'], axis = 1).sum(numeric_only=True, axis = 1)
inventaire_ges['total'] = inventaire_ges['total'].fillna(inventaire_ges['total'].mean(numeric_only=True, axis = 0))
inventaire_ges['log_total'] = np.log(inventaire_ges['total'])

#On spatialise le DataFrame à l'aide du jeu de donnée de l'INSEE
#Pour la compatibilité avec l'INSEE
inventaire_ges = inventaire_ges.rename(columns={'Commune':'nom_m'})
ademe_gdf = gpd.GeoDataFrame(
    pd.merge(inventaire_ges,mapcom, how = 'left', on = 'nom_m', validate='m:m', suffixes=(False,False)),
    geometry='geometry'
)

ademe_gdf.sample(5)

,INSEE commune,nom_m,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,...,insee_com,statut,population,insee_can,insee_arr,insee_dep,insee_reg,siren_epci,geometry,crsCoord
24980,53223,SAINT-GERMAIN-DE-COULAMER,6938.003261,NaN,NaN,275.132683,123.053406,2.354558,6.911213,193.599667,...,53223,Commune simple,329.0,17,3,53,52,200042182,"MULTIPOLYGON (((-0.16132 48.23898, -0.16185 48...",EPSG:3857
5757,14147,CERNAY,1427.888031,NaN,NaN,146.469107,20.233198,2.354558,6.911213,78.231058,...,86047,Commune simple,491.0,08,1,86,75,248600413,"MULTIPOLYGON (((0.30093 46.84914, 0.30099 46.8...",EPSG:3857
5632,14021,ARROMANCHES-LES-BAINS,1222.334411,NaN,NaN,502.422993,71.411287,18.836462,55.289707,345.531314,...,14021,Commune simple,449.0,11,1,14,28,241400555,"MULTIPOLYGON (((-0.61091 49.34016, -0.61156 49...",EPSG:3857
32978,67060,BOURGHEIM,913.637901,16.262204,NaN,500.996998,74.188393,2.354558,6.911213,323.844245,...,67060,Commune simple,635.0,12,5,67,44,200034270,"MULTIPOLYGON (((7.46229 48.4246, 7.46238 48.42...",EPSG:3857
31529,64083,AUTEVIELLE-SAINT-MARTIN-BIDEREN,381.989653,NaN,NaN,178.210365,100.307589,2.354558,6.911213,65.530483,...,64083,Commune simple,207.0,16,2,64,75,200067288,"MULTIPOLYGON (((-0.95743 43.39744, -0.95784 43...",EPSG:3857


## 1.4. Jonction Geod'air-MétéoFrance

Dans cette partie on va procéder à la jonction entre les jeux de donnée Géod'air et météofrance. Ici un simple Merge ou un sjoin ne suffisent pas car nous ne pouvons pas assumer priori qu'il y ait une station de mesure météofrance pour chaque station de mesure utilisée par Géod'air (il y a a priori plus de stations de mesure Géod'air). L'idée est donc d'attacher à chaque mesure de polluant la mesure météofrance du même jour de la station la plus proche.

In [244]:

# On s'assure que les deux GeoDataFrames utilisent le même système de coordonnées
polluants_gdf = polluants_gdf.to_crs("EPSG:4326")
meteo_gdf = meteo_gdf.to_crs("EPSG:4326")
stations_gdf = stations_gdf.to_crs("EPSG:4326")

# On joint d'abord les mesures de polluants avec leur station météofrance la plus proche
polluants_stations_gdf = gpd.sjoin_nearest(
    polluants_gdf,
    stations_gdf,
    how = 'left',
    distance_col="distance_p2m", 
    exclusive=True
    ).to_crs("EPSG:4326").drop(columns = ['id','Nom','index_right'],inplace = False)

# Puis on combine le DataFrame intermédiaire obtenu avec le DataFrame MétéoFrance
processus_gdf = gpd.GeoDataFrame(
    pd.merge(polluants_stations_gdf,df_meteo, how='left', on=['date','ID'], suffixes = (False,False)).reset_index(),
    geometry='geometry'
).reset_index()
processus_gdf.sample(5)

c:\Users\antoi\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,level_0,index,nom_m,date,insee_com,statut,population,insee_dep,NO2,O3,...,Altitude,distance_p2m,pression_niveau_mer_Pa,var_pression_3h_Pa,direction_vent_moyen_10mn_deg,vitesse_vent_moyen_10mn_m/s,temperature_K,humidite_%,visibilite_horizontale_m,precipitations_24h_mm
479177,479177,479177,SALAISE-SUR-SANNE,2022-08-28,38468,Commune simple,4505,38,72.900000,NaN,...,235,0.468334,101320,0,330,2.500000,293.050000,75,40000,0.000000
516959,516959,516959,TOULOUSE,2023-11-11,31555,Préfecture de région,504078,31,43.350000,68.6,...,151,0.059931,101460,-110,260,3.700000,283.550000,93,21210,7.200000
316478,316478,316478,METZ,2022-04-13,57463,Préfecture,120874,57,48.366667,90.7,...,336,0.582032,101330,90,200,4.300000,287.950000,46,60000,0.000000
257428,257428,257428,LE PONTET,2022-01-15,84092,Commune simple,17551,84,77.000000,NaN,...,73,0.633229,103010,-120,40,2.900000,273.750000,80,8970,0.000000
165206,165206,165206,EVREUX,2023-06-26,27229,Préfecture,47289,27,7.200000,84.5,...,151,0.363194,101870,110,280,5.900000,289.550000,83,28560,0.000000


# 2. Analyse exploratoire de données

# 3. Modélisation des séries temporelles

## 3.1. Explications

## 3.2. Préparation des données

In [107]:
# Enregistrement des convertisseurs pour éviter les avertissements
pd.plotting.register_matplotlib_converters()
plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)



One sélectionne les variables d'intérêt, c'est-à-dire les variables stochastiques numériques et les clées de ces variables.

In [108]:
processus_gdf.columns

Index(['level_0', 'index', 'nom_m', 'date', 'NO2', 'geometry', 'O3', 'PM2.5',
       'PM10', 'SO2', 'insee_com', 'statut', 'population', 'insee_dep', 'ID',
       'Latitude', 'Longitude', 'Altitude', 'distance_p2m',
       'pression_niveau_mer_Pa', 'var_pression_3h_Pa',
       'direction_vent_moyen_10mn_deg', 'vitesse_vent_moyen_10mn_m/s',
       'temperature_K', 'humidite_%', 'visibilite_horizontale_m',
       'precipitations_24h_mm'],
      dtype='object')

In [245]:
X_t = processus_gdf.loc[:,[
    'date',
    'nom_m',
    'pression_niveau_mer_Pa',
    'var_pression_3h_Pa',
    'direction_vent_moyen_10mn_deg',
    'vitesse_vent_moyen_10mn_m/s',
    'temperature_K',
    'humidite_%',
    'visibilite_horizontale_m',
    'precipitations_24h_mm'
    ]].reset_index()

X_t.replace('mq', np.nan, inplace=True)

X_t = X_t.astype({
    'pression_niveau_mer_Pa' : 'float',
    'var_pression_3h_Pa' : 'float',
    'direction_vent_moyen_10mn_deg' : 'float',
    'vitesse_vent_moyen_10mn_m/s' : 'float',
    'temperature_K' : 'float',
    'humidite_%' : 'float',
    'visibilite_horizontale_m' : 'float',
    'precipitations_24h_mm'  : 'float'
})

X_t['date'] = pd.to_datetime(X_t['date'], errors='coerce')

X_t.sample(5)

,index,date,nom_m,pression_niveau_mer_Pa,var_pression_3h_Pa,direction_vent_moyen_10mn_deg,vitesse_vent_moyen_10mn_m/s,temperature_K,humidite_%,visibilite_horizontale_m,precipitations_24h_mm
321906,321906,2020-04-09,MONT-DE-MARSAN,102150.0,-20.0,0.0,0.0,284.45,87.0,41370.0,-0.1
101701,101701,2021-02-15,CARTIGNIES,102680.0,-140.0,150.0,8.1,274.55,61.0,23370.0,0.0
132512,132512,2022-07-24,CLERMONT-FERRAND,102130.0,40.0,260.0,1.5,289.95,80.0,38570.0,0.0
512323,512323,2023-03-03,THANN,102100.0,-70.0,160.0,1.4,273.35,87.0,4700.0,0.0
21169,21169,2022-02-10,APT,103100.0,-60.0,110.0,1.1,281.25,87.0,13580.0,0.0


In [246]:
Y_t = processus_gdf.loc[:,[
    'date',
    'nom_m',
    'NO2',
    'O3',
    'PM2.5',
    'PM10'
    ]].reset_index()

Y_t.replace('NaN', np.nan, inplace=True,)

Y_t['date'] = pd.to_datetime(X_t['date'], errors='coerce')

Y_t.dtypes
# Y_t = processus_gdf[['date','nom_m','NO2','O3','PM2.5','PM10']]

index             int64
date     datetime64[ns]
nom_m            object
NO2             float64
O3              float64
PM2.5           float64
PM10            float64
dtype: object

## 3.3 Modélisation

In [ ]:
def eval_model(df, P = 3, Q = 3, R = 3):
  '''Une fonction pour évaluer les modèles SARIMAX'''

  models = pd.DataFrame(np.zeros((R,2), dtype=float), columns = ['AIC','Model'])
  warnings.simplefilter('ignore')
  # Itération des modèles ARMA(p,q,r)
  for r in range(R):
    aic = pd.DataFrame(np.zeros((P,Q), dtype=float))
    aic.iloc[0,0] = np.nan
    for p in range(P):
      for q in range(Q):
        if p == 0 and q == 0:
          continue

        # Estimation de chaque model
        mod = sm.tsa.statespace.SARIMAX(df, order=(p,r,q), enforce_invertibility=False)
        try:
          res = mod.fit(disp=False)
          aic.iloc[p,q] = res.aic
        except:
          aic.iloc[p,q] = np.nan

        models['AIC'][r] = aic.min().min()
        coord_min = aic.stack().idxmin()
        models['Model'][r] = str(coord_min[0])+str(r) + str(coord_min[1])
  return models

def select_model(df, P = 3, Q = 3, R = 3):
  '''Une fonction pour sélectionner le meilleur modèle SARIMAX'''
  df_eval = eval_model(df,P,Q,R)
  best_model = df_eval['Model'][df_eval['AIC'].idxmin()]
  return best_model


In [248]:
liste_communes = Y_t['nom_m'].unique()
#print(liste_communes) # Le jeu de données ne contient aucune données sur Palaiseau :(
# On va prendre 'STRASBOURG' pour exemples
Y_t[(Y_t['nom_m'] == 'STRASBOURG')]

,index,date,nom_m,NO2,O3,PM2.5,PM10
501067,501067,2020-01-01,STRASBOURG,47.850,4.9,58.50,84.50
501068,501068,2020-01-02,STRASBOURG,40.075,9.3,14.50,24.50
501069,501069,2020-01-03,STRASBOURG,40.675,44.9,10.05,15.00
501070,501070,2020-01-04,STRASBOURG,51.625,62.9,7.45,14.75
501071,501071,2020-01-05,STRASBOURG,79.650,37.8,16.00,25.75
...,...,...,...,...,...,...,...
502524,502524,2023-12-28,STRASBOURG,44.675,51.5,6.40,10.60
502525,502525,2023-12-29,STRASBOURG,28.100,65.9,5.20,8.00
502526,502526,2023-12-30,STRASBOURG,49.200,63.9,9.40,11.00
502527,502527,2023-12-31,STRASBOURG,24.700,71.9,11.00,11.20
